In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import numpy as np
import pandas as pd

In [2]:
def corr2d(X,K):
    n_h,n_w = X.shape
    k_h,k_w = K.shape
    Y = torch.zeros(n_h - k_h +1, n_w - k_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = d2l.reduce_sum(X[i:i+k_h,j:j+k_w] * K)
    return Y

# def corr2d(X, K):  #@save
#     """Compute 2D cross-correlation."""
#     h, w = K.shape
#     Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
#     for i in range(Y.shape[0]):
#         for j in range(Y.shape[1]):
#             Y[i, j] = d2l.reduce_sum((X[i: i + h, j: j + w] * K))
#     return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
flipped_K = torch.flip(K,[0,1])
print(K)
print(flipped_K)
print(corr2d(X,K))
print(corr2d(X,flipped_K))

tensor([[0., 1.],
        [2., 3.]])
tensor([[3., 2.],
        [1., 0.]])
tensor([[19., 25.],
        [37., 43.]])
tensor([[ 5., 11.],
        [23., 29.]])


In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### Creating a con layer to learn

In [6]:
X = torch.ones(6,8)
X[:,2:6] = 0
K = torch.tensor([[1,-1]])
Y = corr2d(X,K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [7]:
conv2d = nn.Conv2d(1,1,K.shape,bias= False)
X = X.reshape(1,1,6,8)
Y = Y.reshape(1,1,6,7)
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 15.944
batch 4, loss 4.776
batch 6, loss 1.662
batch 8, loss 0.631
batch 10, loss 0.250


In [8]:
d2l.reshape(conv2d.weight.data, (1, 2))

tensor([[ 0.9349, -1.0371]])